Note that I have already selected the top 500 features in terms of variance.

However, this is still too many features and simply taking the top 10 most varying features can be biased (could be that the difference in variance is not so great).

Thus, I have decided to use RFEVC to select the top 15 features.

In [2]:
import sys
sys.path.append("/Users/shamanthk/Documents/iomics/dowhy-preeclampsia")

In [3]:
import numpy as np
import pandas as pd
from EstimateEffect import *

No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.


In [4]:
betas_df = pd.read_csv('data/betas.csv')
pds_df = pd.read_csv('data/pds.csv')

df = preproc(betas_df, pds_df)

df.head()

NameError: name 'preproc' is not defined

In [ ]:
df.shape

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
from sklearn.ensemble import RandomForestClassifier

X = df.drop(columns=['Group'])
y = df['Group']

Method 1: Chi-squared

Evaluates the independence between each feature and the categorical target.

Compares observed values in the data with the expected values assuming independence

$$\chi^2 = \sum \frac{(O_i-E_i)^2}{E_i}$$

$O_i \rightarrow$ Observed frequency

$E_i \rightarrow$ Expected frequency 

The higher the score, the more likely that the feature is dependent on the target.

Strengths:
- Very fast and interpretable.

Limitations:
- Ignores relationships between features.
- Requires features and target to be discrete.

In [ ]:
chi2_selector = SelectKBest(score_func=chi2, k=50)
chi2_selector.fit(X, y)
chi2_selected_features = X.columns[chi2_selector.get_support()]
print("Top 15 features (Chi-square):", chi2_selected_features.tolist())

Top 15 features (Chi-square): ['ASAP1', 'ATP13A4', 'BCAR1', 'BCL11B', 'C4BPB', 'CCDC57', 'CDK14', 'CEP110', 'CHI3L2', 'CYP11A1', 'DAPK3', 'EFEMP1', 'EFS', 'GABRA1', 'GATA2', 'GCOM1', 'GLRX', 'GNA13', 'GPD1L', 'GPR135', 'IL17F', 'KRT24', 'LBP', 'LIPH', 'LVRN', 'MC1R', 'MIR1204', 'MLX', 'MPZL1', 'NDUFAF3', 'NELL1', 'OR51B4', 'PAPPA2', 'PAX7', 'PCDH17', 'PLCB1', 'PRDM14', 'PVRL4', 'SERPINE1', 'SFT2D3', 'SLC13A4', 'SLC19A1', 'SNX7', 'SPERT', 'ST6GALNAC1', 'TIAM1', 'TMPRSS11F', 'TNFRSF10C', 'TSPAN4', 'VPS4B']


Method 2: Mutual Information

Measures how much knowing a feature reduces the uncertainty of the target.

Quantifies the amount of information shared between two variables.

$$MI(X, Y) = \sum_{x\in X}\sum_{y\in Y} P(x, y) \log(\frac{P(x,y)}{P(x)P(y)})$$

$P(x,y) \rightarrow$ joint probability of x and y

$P(x), P(y) \rightarrow$ marginal probabilities of x and y, respectively

A higher $MI$ represents a stronger dependency

Strengths:
- Captures non-linear relationships.
- Suitable for both discrete and continuous features.

Limitations:
- MI values are non-negative but unbounded, making direct comparison tricky.
- Computationally heavier than Chi-square for large datasets.

In [ ]:
mi_selector = SelectKBest(score_func=mutual_info_classif, k=50)
mi_selector.fit(X, y)
mi_selected_features = X.columns[mi_selector.get_support()]
print("Top 15 features (Mutual Information):", mi_selected_features.tolist())

Top 15 features (Mutual Information): ['A2ML1', 'ALOX5AP', 'ANXA9', 'C4BPB', 'CA1', 'CCDC27', 'CD96', 'CEP110', 'CHI3L2', 'CLCA1', 'CNR2', 'CYP11A1', 'CYP2F1', 'DAPK3', 'DDB1', 'DUSP13', 'ELF5', 'FAM110A', 'FXYD3', 'GNA13', 'HERPUD1', 'HSD17B1', 'ICMT', 'IL17F', 'ITIH4', 'KRT24', 'KRT8', 'LIPH', 'LOC55908', 'MC2R', 'MMP19', 'MPZL1', 'MTNR1B', 'MYO3A', 'NAF1', 'PAPPA2', 'PLEKHF1', 'PTPN6', 'PUM2', 'PVRL4', 'RAPGEFL1', 'RDH5', 'REPIN1', 'SGK2', 'SLC13A4', 'SLC19A1', 'TGM6', 'TIAM1', 'TMPRSS11F', 'YPEL4']


Method 3: RF Feature Importance

Identifies important features by measuring their contribution to reducing impurity in decision trees.

In [ ]:
rf = RandomForestClassifier(random_state=52)
rf.fit(X, y)
feature_importances = rf.feature_importances_
rf_top_indices = feature_importances.argsort()[-50:][::-1]
rf_top_features = X.columns[rf_top_indices]
print("Top 15 features (Random Forest):", rf_top_features.tolist())

Top 15 features (Random Forest): ['DAPK3', 'SLC13A4', 'A2ML1', 'TMPRSS11F', 'ICMT', 'MPZL1', 'FXYD3', 'CYP11A1', 'SLC19A1', 'FAM110A', 'DNASE1L3', 'PAPPA2', 'DUSP13', 'CSF3R', 'DNAH5', 'MMP19', 'SLC36A2', 'KRT31', 'REPIN1', 'DEGS1', 'YPEL4', 'PIK3C2A', 'BMP10', 'GNA13', 'RCBTB2', 'DDB1', 'DQX1', 'BTN1A1', 'GAL3ST1', 'CEP110', 'C4BPB', 'PCDH12', 'PUM2', 'PAK6', 'DCD', 'KRT19', 'IPCEF1', 'ALOX5AP', 'KISS1', 'CD27', 'IL17F', 'PLA2G1B', 'IL1RL1', 'AP1G2', 'CER1', 'TIAM1', 'SPERT', 'SFT2D3', 'CARD8', 'C6orf25']


Checking the intersecting features -- those that are found to be important by all the three methods

In [ ]:
intersection = set(chi2_selected_features) & set(mi_selected_features) & set(rf_top_features)
list(intersection)

['C4BPB',
 'TMPRSS11F',
 'SLC13A4',
 'MPZL1',
 'SLC19A1',
 'GNA13',
 'CYP11A1',
 'IL17F',
 'DAPK3',
 'CEP110',
 'TIAM1',
 'PAPPA2']

In [ ]:
df = df.loc[:, list(intersection)]
df.shape

NameError: name 'df' is not defined